In [2]:
# import package
import os
import glob
import pandas as pd
import numpy as np
import warnings
from IPython.display import FileLink
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 300)

In [4]:
# Data directory
data_dir = '/Users/jerry/Library/CloudStorage/OneDrive-TheChineseUniversityofHongKong/University Courses/STAT (MAJOR)/stat 4011 (in progress)/part 2/original data/'

In [6]:
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

In [8]:
# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('/Users/jerry/Library/CloudStorage/OneDrive-TheChineseUniversityofHongKong/University Courses/STAT (MAJOR)/stat 4011 (in progress)/part 2/original data/train.csv')
    test = pd.read_csv('/Users/jerry/Library/CloudStorage/OneDrive-TheChineseUniversityofHongKong/University Courses/STAT (MAJOR)/stat 4011 (in progress)/part 2/original data/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

In [10]:
# Function to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train=True):
    df_list = []
    
    for stock_id in list_stock_ids:
        
        print(f"Processing stock_id: {stock_id}")
        
        if is_train:
            file_path_book = f"{data_dir}book_train.parquet/stock_id={stock_id}"
            file_path_trade = f"{data_dir}trade_train.parquet/stock_id={stock_id}"
        else:
            file_path_book = f"{data_dir}book_test.parquet/stock_id={stock_id}"
            file_path_trade = f"{data_dir}trade_test.parquet/stock_id={stock_id}"

        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on='row_id', how='left')
        df_list.append(df_tmp)

    df = pd.concat(df_list, ignore_index=True)
    return df

In [12]:
# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return).reset_index(drop=True)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return).reset_index(drop=True)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread': [np.sum, np.mean, np.std],
        'bid_spread': [np.sum, np.mean, np.std],
        'ask_spread': [np.sum, np.mean, np.std],
        'total_volume': [np.sum, np.mean, np.std],
        'volume_imbalance': [np.sum, np.mean, np.std]
    }

    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix=False):
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket=0, add_suffix=False)
    df_feature_450 = get_stats_window(seconds_in_bucket=450, add_suffix=True)
    df_feature_300 = get_stats_window(seconds_in_bucket=300, add_suffix=True)
    df_feature_150 = get_stats_window(seconds_in_bucket=150, add_suffix=True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_450, how='left', left_on='time_id_', right_on='time_id__450')
    df_feature = df_feature.merge(df_feature_300, how='left', left_on='time_id_', right_on='time_id__300')
    df_feature = df_feature.merge(df_feature_150, how='left', left_on='time_id_', right_on='time_id__150')
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis=1, inplace=True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis=1, inplace=True)
    return df_feature

In [14]:
# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return).reset_index(drop=True)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return': [realized_volatility],
        'seconds_in_bucket': [count_unique],
        'size': [np.sum],
        'order_count': [np.mean],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix=False):
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket=0, add_suffix=False)
    df_feature_450 = get_stats_window(seconds_in_bucket=450, add_suffix=True)
    df_feature_300 = get_stats_window(seconds_in_bucket=300, add_suffix=True)
    df_feature_150 = get_stats_window(seconds_in_bucket=150, add_suffix=True)

    # Merge all
    df_feature = df_feature.merge(df_feature_450, how='left', left_on='time_id_', right_on='time_id__450')
    df_feature = df_feature.merge(df_feature_300, how='left', left_on='time_id_', right_on='time_id__300')
    df_feature = df_feature.merge(df_feature_150, how='left', left_on='time_id_', right_on='time_id__150')
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis=1, inplace=True)

    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis=1, inplace=True)
    return df_feature

In [16]:
# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 
                'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 
                'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 
                'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the time id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')

    # Merge with original dataframe
    df = df.merge(df_stock_id, how='left', left_on=['stock_id'], right_on=['stock_id__stock'])
    df = df.merge(df_time_id, how='left', left_on=['time_id'], right_on=['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis=1, inplace=True)
    return df

In [18]:
# Read train and test
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()

Our training set has 428932 rows


In [20]:
# Preprocess them using Parallel and our single stock id functions
print("Start preprocessing training data------------------------------")
train_ = preprocessor(train_stock_ids, is_train=True)
train = train.merge(train_, on=['row_id'], how='left')
print("Finish preprocessing training data-----------------------------")

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
print("Start preprocessing testing data-------------------------------")
test_ = preprocessor(test_stock_ids, is_train=False)
test = test.merge(test_, on=['row_id'], how='left')
print("Finish preprocessing testing data------------------------------")

Start preprocessing training data------------------------------
Processing stock_id: 0
Processing stock_id: 1
Processing stock_id: 2
Processing stock_id: 3
Processing stock_id: 4
Processing stock_id: 5
Processing stock_id: 6
Processing stock_id: 7
Processing stock_id: 8
Processing stock_id: 9
Processing stock_id: 10
Processing stock_id: 11
Processing stock_id: 13
Processing stock_id: 14
Processing stock_id: 15
Processing stock_id: 16
Processing stock_id: 17
Processing stock_id: 18
Processing stock_id: 19
Processing stock_id: 20
Processing stock_id: 21
Processing stock_id: 22
Processing stock_id: 23
Processing stock_id: 26
Processing stock_id: 27
Processing stock_id: 28
Processing stock_id: 29
Processing stock_id: 30
Processing stock_id: 31
Processing stock_id: 32
Processing stock_id: 33
Processing stock_id: 34
Processing stock_id: 35
Processing stock_id: 36
Processing stock_id: 37
Processing stock_id: 38
Processing stock_id: 39
Processing stock_id: 40
Processing stock_id: 41
Processing

In [21]:
# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

In [22]:
# Download new testing data
test.to_pickle('new_test.pkl')
download_test = FileLink('new_test.pkl')
download_test

/Users/jerry/Library/CloudStorage/OneDrive-TheChineseUniversityofHongKong/University Courses/STAT (MAJOR)/stat 4011 (in progress)/part 2/new_test.pkl

In [23]:
# Download new training data
train.to_pickle('new_train.pkl')
download_train = FileLink('new_train.pkl')
download_train

/Users/jerry/Library/CloudStorage/OneDrive-TheChineseUniversityofHongKong/University Courses/STAT (MAJOR)/stat 4011 (in progress)/part 2/new_train.pkl

In [35]:
train

,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,wap_balance_sum,wap_balance_mean,wap_balance_std,price_spread_sum,price_spread_mean,price_spread_std,bid_spread_sum,bid_spread_mean,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_std,total_volume_sum,total_volume_mean,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,wap1_sum_450,wap1_mean_450,wap1_std_450,wap2_sum_450,wap2_mean_450,wap2_std_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,wap_balance_sum_450,wap_balance_mean_450,wap_balance_std_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,wap1_sum_300,wap1_mean_300,wap1_std_300,wap2_sum_300,wap2_mean_300,wap2_std_300,log_return1_sum_300,log_return1_realized_volatility_300,log_return1_mean_300,log_return1_std_300,log_return2_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,wap_balance_sum_300,wap_balance_mean_300,wap_balance_std_300,price_spread_sum_300,price_spread_mean_300,price_spread_std_300,bid_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,ask_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,total_volume_sum_300,total_volume_mean_300,total_volume_std_300,volume_imbalance_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,wap1_sum_150,wap1_mean_150,wap1_std_150,wap2_sum_150,wap2_mean_150,wap2_std_150,log_return1_sum_150,log_return1_realized_volatility_150,log_return1_mean_150,log_return1_std_150,log_return2_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,wap_balance_sum_150,wap_balance_mean_150,wap_balance_std_150,price_spread_sum_150,price_spread_mean_150,price_spread_std_150,bid_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,ask_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,total_volume_sum_150,total_volume_mean_150,total_volume_std_150,volume_imbalance_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return1_realized_volatility_450_mean_stock,log_return1_realized_volatility_450_std_stock,log_return1_realized_volatility_450_max_stock,log_return1_realized_volatility_450_min_stock,log_return2_realized_volatility_450_mean_stock,log_return2_realized_volatility_450_std_stock,log_return2_realized_volatility_450_max_stock,log_return2_realized_volatility_450_min_stock,log_return1_realized_volatility_300_mean_stock,log_return1_realized_volatility_300_std_stock,log_return1_realized_volatility_300_max_stock,log_return1_realized_volatility_300_min_stock,log_return2_realized_volatility_300_mean_stock,log_return2_realized_volatility_300_std_